In [ ]:
# Input information
# NOTE: the file name is just a file the author used for testing, the file is not included in the repository

# SA2-141E
# meeting = {
#     'file':  "2020.10.22 080146 email summary.csv",
#     'title': "SA2-141E",
#     'dates': ['2020-10-12 00:00','2020-10-22 10:00'],
#     'tdocs_cache': 'TDocsByAgenda_TSGS2_141e_Electronic_745f0abc6d641cb65afd5d8aef4476a7.pickle'
# }

# SA2-140E
# meeting = {
#     'file':  "2020.09.02 090903 email summary.csv",
#     'title': "SA2-140E",
#     'dates': ['2020-08-19 12:00','2020-08-31 22:00'],
#     'tdocs_cache': 'TDocsByAgenda_TSGS2_140e_Electronic_376bc93cdb21add4d0c7abe101f8f232.pickle'
# }

# SA2-139E Ph1+Ph2
# meeting = {
#     'file':  "2020.06.15 073045 email summary.csv",
#     'title': "SA2-139E Ph1+Ph2",
#     'dates': ['2020-05-31 12:00','2020-06-12 19:00']
# }

# SA2-138E
#meeting = {
#    'file':  "2020.04.24 153334 email summary.csv",
#    'title': "SA2-138E",
#    'dates': ['2020-04-19 12:00','2020-04-24 19:00']
#}

# SA2-137E
meeting = {
   'file':  "2020.03.02 080314 email summary.csv",
   'title': "SA2-137E",
   'dates': ['2020-02-23 12:00','2020-02-28 19:00'],
   'tdocs_cache': 'TDocsByAgenda_TSGS2_137e_Electronic_f758bcb52ead1c245c3797d880ae986c.pickle'
}

In [ ]:
# You will need pandas/numpy to run the first part of the script. Then plotly for the second part. You can install plotly via pip (pip install plotly)
# If the PNG export is not working, most probably orca is not installed. You can install it by executing in the Anaconda cmd: conda install -c plotly plotly-orca
import pandas as pd
import numpy as np
import plotly.express as px
import re
import pickle
import traceback

In [ ]:
# Create variables
email_dump    = meeting["file"]
title         = meeting["title"]
meeting_dates = meeting["dates"]

# Create DataFrame for analysis
df = pd.read_csv(
    email_dump,
    index_col=0,
    dtype=object)
df['date']    = pd.to_datetime(df['date'])
df['day']     = df['date'].dt.floor('d')
df['hour']    = df['date'].dt.floor('h')

def parse_email_company(x):
    try:
        email_address_split = x.split('@')
        company_domain      = email_address_split[-1]
        company_name_split  = company_domain.split('.')
        
        if 'COM.CN' in x:
            company_name = company_name_split[0]
        else:
            company_name = company_name_split[-2]
        
        if len(company_name) <= 4:
            company_name = company_name.upper()
        else:
            company_name = company_name.capitalize()
            
        if company_name=='COM':
            print(x)
            
        # Some substitutions for big companies that show in rankings
        if company_name=='Ztetx':
            company_name = 'ZTE'
        elif company_name=='Nokia-sbell' or company_name=='Nokia-bell-labs':
            company_name = 'Nokia'
        elif company_name=='Docomolab-euro':
            company_name = 'NTT Docomo'
        elif company_name=='Magenta':
            company_name = 'Deutsche Telekom'
        elif company_name=='Futurewei':
            company_name = 'Huawei'
            
        return company_name
    except:
        print('Error parsing email. Sender address: {0}'.format(x))
        return ''
df['company'] = df['sender address'].apply(parse_email_company)

display(df.company.unique())

In [ ]:
df

In [ ]:
email_count_per_ai   = df.groupby('ai').count()
email_count_per_day  = df.groupby('day').count()
email_count_per_hour = df.groupby('hour').count()

In [ ]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count = df.groupby('sender name').count().sort_values(by='date', ascending=False)
# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count['ranking'] = np.arange(len(emails_count)) + 1
emails_count['emails'] = emails_count['date']
display(emails_count[['ranking', 'emails']].iloc[0:25,:])
pd.options.display.max_rows = 60

In [ ]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count_company = df.groupby('company').count().sort_values(by='date', ascending=False)

# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count_company['ranking'] = np.arange(len(emails_count_company)) + 1
emails_count_company['emails'] = emails_count_company['date']
display(emails_count_company[['ranking', 'emails']].iloc[0:20,:])
pd.options.display.max_rows = 60

In [ ]:
# Ranking based on emails per delegate
active_delegate_list = pd.DataFrame()
grouped_df = df.groupby(['company', 'sender name']).count()
active_delegate_list['company']       = grouped_df.index.get_level_values(0).copy()
active_delegate_list['delegate name'] = grouped_df.index.get_level_values(1).copy()
# display(active_delegate_list)
active_delegate_count = active_delegate_list.groupby('company').count()
# display(active_delegate_count)
# display(emails_count_company)

emails_per_active_delegate_count = emails_count_company['emails']/active_delegate_count['delegate name']
emails_count_company['emails/active delegate'] = emails_per_active_delegate_count
emails_count_company['active delegates'] = active_delegate_count['delegate name']
emails_count_company['total emails'] = emails_count_company['emails']
emails_count_company_resort = emails_count_company.sort_values(by='emails/active delegate', ascending=False)
emails_count_company_resort['ranking'] = np.arange(len(emails_count_company)) + 1
display(emails_count_company_resort[['ranking', 'emails/active delegate', 'active delegates', 'total emails']].iloc[0:20,:])

In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
data = go.Bar(
    x=email_count_per_ai.index, 
    y=email_count_per_ai['date'])
layout = go.Layout(
    xaxis= {'type': 'category'},
    title='{1}: Emails/AI (total {0})'.format(email_count_per_ai['date'].sum(), title))
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
data = go.Bar(
    x=email_count_per_day.index, 
    y=email_count_per_day['date'])
layout = go.Layout(
    title='Emails per day (total {0})'.format(email_count_per_ai['date'].sum()),
    xaxis_tickformat = '%d.%m',
    xaxis_range = meeting_dates,
    xaxis = { 'dtick': 1000*3600*24 })
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per day")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Data source for emails/day per AI
email_count_per_day_and_ai = df.groupby(['ai', 'day']).count()

ai_plot = {}
ais = df.ai.unique()
for ai in ais:
    ai_plot[ai] = email_count_per_day_and_ai[email_count_per_day_and_ai.index.isin([ai], level=0)]['date']

layout = go.Layout(
    title='{1}: Emails/day (total {0})'.format(email_count_per_ai['date'].sum(), title),
    xaxis_tickformat = '%d.%m',
    xaxis_range = meeting_dates,
    xaxis = { 'dtick': 1000*3600*24 } )

fig = go.Figure(layout=layout)

for ai in ais:
    bar_data = ai_plot[ai]
    bar_y = bar_data
    bar_x = bar_data.index.get_level_values(1)
    bar_title = ai
    data = go.Bar(
        x=bar_x, 
        y=bar_y,
        name=bar_title)
    fig.add_trace(data)

fig.update_layout(barmode='relative')
fig.show()

fig_title = "{0} {1}".format(title, "Emails per day and AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Deactivated cell (we already have the plot per AI)
data = go.Bar(
    x=email_count_per_hour.index, 
    y=email_count_per_hour['date'])
layout = go.Layout(
    title='Emails per hour (total {0})'.format(email_count_per_ai['date'].sum()),
    xaxis_tickformat = '%d.%m  %Hh',
    xaxis_range = meeting_dates)
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per hour")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Data source for emails/h per AI
email_count_per_hour_and_ai = df.groupby(['ai', 'hour']).count()

ai_plot = {}
ais = df.ai.unique()
for ai in ais:
    ai_plot[ai] = email_count_per_hour_and_ai[email_count_per_hour_and_ai.index.isin([ai], level=0)]['date']

layout = go.Layout(
    title='{1}: Emails/hour (total {0})'.format(email_count_per_ai['date'].sum(), title),
    xaxis_tickformat = '%d.%m %Hh',
    xaxis_range = meeting_dates)

fig = go.Figure(layout=layout)

for ai in ais:
    bar_data = ai_plot[ai]
    bar_y = bar_data
    bar_x = bar_data.index.get_level_values(1)
    bar_title = ai
    data = go.Bar(
        x=bar_x, 
        y=bar_y,
        name=bar_title)
    fig.add_trace(data)

fig.update_layout(barmode='relative')
fig.show()

fig_title = "{0} {1}".format(title, "Emails per hour and AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
tdocs = []    
for email_subject in df['subject']:
    try:
        tdocs.append(re.search(r'S2-[\d]{7}', email_subject).group())
    except:
        tdocs.append('')
        # print(email_subject)

df['tdoc'] = tdocs
# display(df)

In [ ]:
df_per_tdoc = df.groupby("tdoc")
emails_per_tdoc = df_per_tdoc.count()
emails_per_tdoc['email count'] = emails_per_tdoc['date']
emails_per_tdoc = emails_per_tdoc.loc[:,['email count']]
emails_per_tdoc.sort_values(by ='email count' , ascending=False, inplace=True)
emails_per_tdoc = emails_per_tdoc.reset_index()

display(emails_per_tdoc.iloc[0:21,:])

In [ ]:
fig = px.histogram(
    emails_per_tdoc, 
    x="email count",
    labels={
        "email count": "Number of emails",
        "count": "Number of TDocs"
    },)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per discussed TDoc")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

print('Mean: {0}'.format(emails_per_tdoc['email count'].mean()))

In [ ]:
# Cross data with TdocsByAgenda data if data is available
try:
    with open(meeting['tdocs_cache'], 'rb') as f:
        # Unpickle the 'data' dictionary using the highest protocol available.
        cache         = pickle.load(f)
        meeting_tdocs = cache['tdocs']
        # Same order and TDocs as the ones in the emails
        meeting_tdocs = meeting_tdocs.loc[list(emails_per_tdoc['tdoc']),['AI', 'Title', 'Source']]
        # display(meeting_tdocs)
        # Need to convert to list. If not you get a list of NaNs because Pandas will try to align the indices
        pd.options.display.max_columns = None
        pd.options.display.max_rows    = 50
        pd.options.display.min_rows    = 20
        meeting_tdocs['email count'] = list(emails_per_tdoc['email count'])
        meeting_tdocs.sort_values(by ='email count' , ascending=False, inplace=True)
        
        # Search for NaNs
        nan_entries     = list(meeting_tdocs[meeting_tdocs['AI'].isnull()].index)
        # print(nan_entries)
        for nan_entry in nan_entries:
            try:
                email_subject = df[df['subject'].str.contains("{0}] ".format(nan_entry))].iloc[0]['subject']
                split1 = email_subject.split('] ')
                meeting_tdocs.at[nan_entry,'Title'] = split1[-1]
                split2 = split1[0].split(', ')[1]
                meeting_tdocs.at[nan_entry,'AI'] = split2.replace('AI#','')
                # print(emails_for_nans)
            except:
                print("Error on {0}".format(nan_entry))
        
        display(meeting_tdocs.iloc[0:21,:])
        # display(meeting_tdocs)
except:
    meeting_tdocs = None
    traceback.print_exc()